In [1]:
import numpy as np
import pandas as pd
import random

# --- CONFIGURATION ---
NUM_SWITCHES = 20  # 4 Core, 8 Agg, 8 Edge (Standard Fat-Tree 16)
NUM_HOSTS = 16
LINKS_PER_SWITCH = 4 # k=4 Fat Tree
CAPACITY = 100.0 # Link Capacity in Mbps (Simulated)

class MockRyuController:
    """
    This class mimics the Ryu Controller API.
    The RL Agent will interact with THIS, thinking it's the real network.
    """
    def __init__(self, traffic_file="synthetic_traffic_16hosts.csv"):
        # 1. Load the WGAN Traffic
        self.traffic_data = pd.read_csv(traffic_file, header=None).values
        self.time_step = 0
        self.max_steps = self.traffic_data.shape[0]
        
        # 2. Build the "Virtual Switch" State
        # We map (SwitchID, PortID) -> Current Bandwidth Usage
        self.switch_stats = {} 
        self._reset_stats()
        
        # 3. Define the Topology (Simplified k=4 Fat-Tree for simulation)
        # We need this to know that "Port 2 on Switch 1 goes to Switch 3"
        self.topo_map = self._build_fat_tree_topo()

    def _reset_stats(self):
        """Resets all simulated switch counters to 0"""
        for sw_id in range(1, NUM_SWITCHES + 1):
            self.switch_stats[sw_id] = {port: 0.0 for port in range(1, LINKS_PER_SWITCH + 1)}

    def _build_fat_tree_topo(self):
        """
        Creates a dictionary mapping: (Switch, Port) -> (Next_Switch, Next_Port)
        This is a 'Digital Twin' of the Mininet Fat-Tree.
        """
        topo = {}
        # ... (We act as if the topology is static for now)
        # For the mock, we assume simple connectivity
        return topo

    # --- RYU API SIMULATION (The commands your Agent will call) ---

    def get_all_switch_stats(self):
        """
        MOCK of: EventOFPFlowStatsReply
        Returns: A dictionary of {dpid: {port: speed}}
        This is the INPUT (State) for your DL Model.
        """
        # 1. Get the traffic for this second from WGAN
        if self.time_step >= self.max_steps:
            self.time_step = 0 # Loop back
            
        current_traffic = self.traffic_data[self.time_step]
        
        # 2. SIMULATE the network traffic
        # We take the WGAN data (Host-to-Host) and "spray" it onto the switches
        # This creates the "Congestion" the agent sees.
        self._simulate_network_load(current_traffic)
        
        # 3. Return the stats in Ryu format
        return self.switch_stats

    def mod_flow(self, dpid, out_port, priority=1):
        """
        MOCK of: parser.OFPFlowMod()
        The Agent calls this to change a path.
        """
        # In simulation, we 'pretend' this fixed the congestion for the NEXT step.
        # For the mock, we just log it.
        # print(f"  [MockRyu] Switch {dpid}: Rerouting flow to Port {out_port}")
        return True

    def tick(self):
        """Moves the clock forward 1 second (Simulating real time)"""
        self.time_step += 1

    # --- INTERNAL PHYSICS ENGINE (How WGAN traffic fills the switches) ---
    
    def _simulate_network_load(self, traffic_row):
        """
        Takes one row of WGAN traffic (Host-to-Host) and fills the virtual links.
        This is the 'Physics' of the simulation.
        """
        self._reset_stats()
        
        # Reshape to matrix
        t_matrix = traffic_row.reshape(NUM_HOSTS, NUM_HOSTS)
        
        # For every pair of hosts, add their traffic to the 'Virtual Links'
        # (Simplified: We just fill random core links to mimic load balancing)
        for src in range(NUM_HOSTS):
            for dst in range(NUM_HOSTS):
                volume = t_matrix[src, dst]
                if volume > 0:
                    # SIMULATION LOGIC:
                    # Traffic from Host A to Host B goes through:
                    # Edge Switch -> Agg Switch -> Core Switch -> Agg -> Edge
                    
                    # 1. Edge Uplink (Port 3 or 4)
                    sw_edge = (src // 2) + 1 # Simple mapping
                    self.switch_stats[sw_edge][3] += volume
                    
                    # 2. Core Link (Randomly congested based on volume)
                    sw_core = random.randint(17, 20) 
                    self.switch_stats[sw_core][1] += volume

# --- TEST THE MOCK ENVIRONMENT ---
print("✅ Mock Ryu Environment Created.")
print("Initializing Controller...")
mock_ryu = MockRyuController()

# Let's see what the Agent sees (The State)
print("\n--- Testing 'get_stats' (Input for DL) ---")
stats = mock_ryu.get_all_switch_stats()

# Print stats for Switch 1 (Edge) and Switch 17 (Core)
print(f"Switch 1 (Edge) Port Usage: {stats[1]}")
print(f"Switch 17 (Core) Port Usage: {stats[17]}")

# Let's try to 'Modify a Flow' (The Action)
print("\n--- Testing 'mod_flow' (Action for DL) ---")
mock_ryu.mod_flow(dpid=1, out_port=2)

✅ Mock Ryu Environment Created.
Initializing Controller...

--- Testing 'get_stats' (Input for DL) ---
Switch 1 (Edge) Port Usage: {1: 0.0, 2: 0.0, 3: np.float64(0.40716129696721), 4: 0.0}
Switch 17 (Core) Port Usage: {1: np.float64(1.4139377793493), 2: 0.0, 3: 0.0, 4: 0.0}

--- Testing 'mod_flow' (Action for DL) ---


True

In [2]:
import time

# --- 1. Initialize the Mock Environment ---
print("🔌 Initializing Mock Ryu Controller...")
try:
    # Ensure 'synthetic_traffic_16hosts.csv' is in the same folder
    mock_ryu = MockRyuController(traffic_file="synthetic_traffic_16hosts.csv")
    print("✅ Controller started successfully.")
except FileNotFoundError:
    print("❌ Error: 'synthetic_traffic_16hosts.csv' not found. Did you run the WGAN export cell?")

# --- 2. The Simulation Loop (Run for 3 Seconds) ---
print("\n--- 🚦 Starting 3-Second Traffic Test ---")

for step in range(3):
    print(f"\n[Time Step {step}] Requesting Switch Stats...")
    
    # A. Get the Stats (The AI's Input)
    # This triggers the 'Physics Engine' to read the CSV and fill the switches
    stats = mock_ryu.get_all_switch_stats()
    
    # B. Inspect Specific Switches
    # Let's look at Edge Switch 1 (connected to Host 1 & 2)
    # and Core Switch 17 (The Backbone)
    edge_traffic = stats[1][3]   # Port 3 on Switch 1
    core_traffic = stats[17][1]  # Port 1 on Switch 17
    
    print(f"   📊 Edge Switch 1 (Port 3): {edge_traffic:.2f} Mbps")
    print(f"   🔥 Core Switch 17 (Port 1): {core_traffic:.2f} Mbps")
    
    # C. Verification Logic
    if edge_traffic == 0 and core_traffic == 0:
        print("   ⚠️ WARNING: No traffic detected. Is the CSV empty or all zeros?")
    else:
        print("   ✅ Traffic detected.")

    # D. Move Time Forward
    mock_ryu.tick() 

print("\n--- Test Complete ---")

🔌 Initializing Mock Ryu Controller...
✅ Controller started successfully.

--- 🚦 Starting 3-Second Traffic Test ---

[Time Step 0] Requesting Switch Stats...
   📊 Edge Switch 1 (Port 3): 0.41 Mbps
   🔥 Core Switch 17 (Port 1): 1.14 Mbps
   ✅ Traffic detected.

[Time Step 1] Requesting Switch Stats...
   📊 Edge Switch 1 (Port 3): 0.23 Mbps
   🔥 Core Switch 17 (Port 1): 1.21 Mbps
   ✅ Traffic detected.

[Time Step 2] Requesting Switch Stats...
   📊 Edge Switch 1 (Port 3): 0.77 Mbps
   🔥 Core Switch 17 (Port 1): 1.08 Mbps
   ✅ Traffic detected.

--- Test Complete ---
